In [ ]:
# truth social 웹사이트에서 주식 종목100개 티커별로 크롤링
import os
import time
import json
import pandas as pd
from datetime import datetime
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException
from webdriver_manager.chrome import ChromeDriverManager

class TruthSocialCrawler:
    """
    Truth Social 웹사이트에서 주식 종목명으로 검색하여 관련 게시물을 크롤링하는 클래스
    """
    
    def __init__(self, headless=False, output_dir="truth_social_data"):
        """
        크롤러 초기화
        
        Args:
            headless (bool): 브라우저를 화면에 표시할지 여부
            output_dir (str): 결과 데이터를 저장할 디렉토리
        """
        self.output_dir = output_dir
        os.makedirs(output_dir, exist_ok=True)
        
        # 크롬 웹드라이버 설정
        chrome_options = Options()
        if headless:
            chrome_options.add_argument("--headless")
        
        chrome_options.add_argument("--no-sandbox")
        chrome_options.add_argument("--disable-dev-shm-usage")
        chrome_options.add_argument("--disable-notifications")
        chrome_options.add_argument("--start-maximized")
        chrome_options.add_argument("--disable-infobars")
        chrome_options.add_argument("--disable-extensions")
        
        # 사용자 에이전트 설정
        chrome_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Safari/537.36")
        
        # 웹드라이버 초기화
        self.driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)
        self.wait = WebDriverWait(self.driver, 10)
        
        # Truth Social URL
        self.base_url = "https://truthsocial.com/"
        
        # 로그인 여부
        self.is_logged_in = False
        
    def login(self, username, password):
        """
        Truth Social에 로그인
        
        Args:
            username (str): 사용자 이름 또는 이메일
            password (str): 비밀번호
            
        Returns:
            bool: 로그인 성공 여부
        """
        try:
            self.driver.get(self.base_url)
            
            # 로그인 버튼 클릭
            login_button = self.wait.until(EC.element_to_be_clickable((By.XPATH, "//a[contains(text(), 'Log in')]")))
            login_button.click()
            
            # 사용자 이름 입력
            username_field = self.wait.until(EC.visibility_of_element_located((By.XPATH, "//input[@type='text']")))
            username_field.clear()
            username_field.send_keys(username)
            
            # 비밀번호 입력
            password_field = self.driver.find_element(By.XPATH, "//input[@type='password']")
            password_field.clear()
            password_field.send_keys(password)
            
            # 로그인 제출 버튼 클릭
            submit_button = self.driver.find_element(By.XPATH, "//button[@type='submit']")
            submit_button.click()
            
            # 로그인 성공 확인
            try:
                self.wait.until(EC.visibility_of_element_located((By.XPATH, "//div[contains(@class, 'timeline')]")))
                self.is_logged_in = True
                print("로그인 성공!")
                return True
            except TimeoutException:
                print("로그인 실패: 타임라인을 찾을 수 없습니다.")
                return False
                
        except Exception as e:
            print(f"로그인 중 오류 발생: {str(e)}")
            return False

    def search_ticker(self, ticker_name):
        """
        특정 주식 종목명으로 검색
        
        Args:
            ticker_name (str): 검색할 주식 종목명
            
        Returns:
            list: 검색 결과에서 추출한 게시물 목록
        """
        posts = []
        
        try:
            # 검색 페이지로 이동
            self.driver.get(f"{self.base_url}search")
            time.sleep(2)
            
            # 검색어 입력
            search_box = self.wait.until(EC.visibility_of_element_located((By.XPATH, "//input[@type='text']")))
            search_box.clear()
            search_box.send_keys(ticker_name)
            search_box.send_keys(Keys.RETURN)
            time.sleep(3)
            
            # 최신 탭 선택 (탭 구현에 따라 달라질 수 있음)
            try:
                latest_tab = self.driver.find_element(By.XPATH, "//a[contains(text(), 'Latest') or contains(text(), '최신')]")
                latest_tab.click()
                time.sleep(2)
            except NoSuchElementException:
                print("최신 탭을 찾을 수 없습니다. 기본 검색 결과로 진행합니다.")
            
            # 게시물 스크롤 및 수집
            last_height = self.driver.execute_script("return document.body.scrollHeight")
            post_count = 0
            max_posts = 100  # 수집할 최대 게시물 수
            
            while post_count < max_posts:
                # 게시물 요소 찾기
                post_elements = self.driver.find_elements(By.XPATH, "//div[contains(@class, 'status') or contains(@class, 'post')]")
                
                # 새로운 게시물 처리
                for post_element in post_elements[post_count:]:
                    try:
                        # 게시물 데이터 추출
                        post_data = self._extract_post_data(post_element, ticker_name)
                        if post_data:
                            posts.append(post_data)
                            post_count += 1
                            
                        if post_count >= max_posts:
                            break
                    except Exception as e:
                        print(f"게시물 추출 중 오류: {str(e)}")
                
                # 페이지 스크롤
                self.driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                time.sleep(2)
                
                # 스크롤 후 높이 확인
                new_height = self.driver.execute_script("return document.body.scrollHeight")
                if new_height == last_height:
                    # 더 이상 로드할 컨텐츠가 없음
                    break
                last_height = new_height
            
            print(f"{ticker_name}에 대해 {len(posts)}개의 게시물을 수집했습니다.")
            
        except Exception as e:
            print(f"{ticker_name} 검색 중 오류 발생: {str(e)}")
        
        return posts
    
    def _extract_post_data(self, post_element, ticker_name):
        """
        게시물 요소에서 데이터 추출
        
        Args:
            post_element: 게시물 웹 요소
            ticker_name (str): 검색한 종목명
            
        Returns:
            dict: 추출된 게시물 데이터
        """
        try:
            # 사용자 이름
            try:
                username = post_element.find_element(By.XPATH, ".//a[contains(@class, 'username')]").text
            except:
                username = "Unknown"
            
            # 게시 시간
            try:
                timestamp_element = post_element.find_element(By.XPATH, ".//time")
                timestamp = timestamp_element.get_attribute("datetime")
                if not timestamp:
                    timestamp = timestamp_element.text
            except:
                timestamp = "Unknown"
            
            # 게시물 내용
            try:
                content = post_element.find_element(By.XPATH, ".//div[contains(@class, 'content')]").text
            except:
                content = ""
            
            # 좋아요 수
            try:
                likes = post_element.find_element(By.XPATH, ".//span[contains(@class, 'likes-count')]").text
                likes = int(likes.replace(",", "").strip()) if likes else 0
            except:
                likes = 0
            
            # 리트루스(리트윗) 수
            try:
                retruth = post_element.find_element(By.XPATH, ".//span[contains(@class, 'retruth-count')]").text
                retruth = int(retruth.replace(",", "").strip()) if retruth else 0
            except:
                retruth = 0
            
            # 댓글 수
            try:
                comments = post_element.find_element(By.XPATH, ".//span[contains(@class, 'comments-count')]").text
                comments = int(comments.replace(",", "").strip()) if comments else 0
            except:
                comments = 0
            
            # 게시물 URL
            try:
                url = post_element.find_element(By.XPATH, ".//a[contains(@href, '/status/')]").get_attribute("href")
            except:
                url = ""
            
            return {
                "ticker": ticker_name,
                "username": username,
                "timestamp": timestamp,
                "content": content,
                "likes": likes,
                "retruth": retruth,
                "comments": comments,
                "url": url,
                "crawled_at": datetime.now().isoformat()
            }
            
        except Exception as e:
            print(f"게시물 데이터 추출 중 오류: {str(e)}")
            return None
    
    def crawl_all_tickers(self, ticker_list):
        """
        여러 종목명에 대해 크롤링 실행
        
        Args:
            ticker_list (list): 크롤링할 종목명 리스트
            
        Returns:
            pd.DataFrame: 수집된 모든 게시물이 저장된 데이터프레임
        """
        all_posts = []
        
        for ticker in ticker_list:
            print(f"\n크롤링 중: {ticker}")
            ticker_posts = self.search_ticker(ticker)
            
            if ticker_posts:
                all_posts.extend(ticker_posts)
                
                # 중간 결과 저장
                self._save_ticker_data(ticker, ticker_posts)
            
            # 요청 간 딜레이
            time.sleep(5)
        
        # 최종 데이터프레임 생성
        if all_posts:
            df = pd.DataFrame(all_posts)
            self._save_final_data(df)
            return df
        else:
            print("수집된 데이터가 없습니다.")
            return pd.DataFrame()
    
    def _save_ticker_data(self, ticker, posts):
        """
        특정 종목의 게시물을 JSON 파일로 저장
        
        Args:
            ticker (str): 종목명
            posts (list): 게시물 목록
        """
        # 파일명에 사용할 종목명 정리
        safe_ticker = ticker.replace(" ", "_").replace(".", "").replace(",", "").replace("&", "and")
        
        # 저장 경로
        file_path = os.path.join(self.output_dir, f"{safe_ticker}_{datetime.now().strftime('%Y%m%d')}.json")
        
        # JSON 파일로 저장
        with open(file_path, 'w', encoding='utf-8') as f:
            json.dump(posts, f, ensure_ascii=False, indent=4)
        
        print(f"{ticker} 데이터가 {file_path}에 저장되었습니다.")
    
    def _save_final_data(self, df):
        """
        모든 데이터를 CSV와 Excel로 저장
        
        Args:
            df (pd.DataFrame): 저장할 데이터프레임
        """
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        
        # CSV 저장
        csv_path = os.path.join(self.output_dir, f"truth_social_stocks_{timestamp}.csv")
        df.to_csv(csv_path, index=False)
        
        # Excel 저장
        excel_path = os.path.join(self.output_dir, f"truth_social_stocks_{timestamp}.xlsx")
        df.to_excel(excel_path, index=False)
        
        print(f"\n모든 데이터가 저장되었습니다:")
        print(f"CSV: {csv_path}")
        print(f"Excel: {excel_path}")
    
    def close(self):
        """
        웹드라이버 종료
        """
        if self.driver:
            self.driver.quit()
            print("웹드라이버가 종료되었습니다.")


# 사용 예시
if __name__ == "__main__":
    # 크롤링할 주식 종목 리스트
    stock_tickers = [
        'NVIDIA', 'Amazon', 'Apple', 'Tesla', 'Alphabet', 'Google', 'Coca-Cola', 
        'Alibaba', 'Cisco Systems', 'AMD', 'Wells Fargo', 'Bank of America', 
        'Microsoft', 'Palantir Technologies', 'Meta', 'Broadcom', 'Walmart', 
        'TSM', 'JP Morgan', 'Chase', 'Uber', 'Exxon Mobil', 'Pfizer', 
        'Ford Motor', 'Intel', 'AT&T', 'Berkshire Hathaway', 'Johnson & Johnson', 
        'Netflix', 'Oracle', 'Procter & Gamble', 'Merck', 'Visa', 'Citigroup', 
        'Nu Holdings', 'Comcast', 'Chevron', 'AbbVie', 'UnitedHealth', 
        'Uber Technologies', 'Lloyds Banking Group', 'Abbott Laboratories', 
        'Novo Nordisk', 'Stellantis', 'Warner Bros. Discovery', 'Costco Wholesale', 
        'Shell', 'Banco Santander', 'Shopify', 'BP', 'JD.com', 'Barrick Gold', 
        'Eli Lilly', 'Petroleo Brasileiro', 'Philip Morris International', 
        'Bristol-Myers Squibb', 'Micron Technology', 'Itau Unibanco Banco Holding', 
        'Nike', 'Salesforce', 'Boeing', 'Ericsson', 'Astrazeneca', 'PDD Holdings', 
        'Walt Disney', 'Robinhood', 'Infosys', 'Full Truck Alliance', 'NextEra Energy', 
        'VALE', 'Boston Scientific', 'Freeport-McMoRan', 'MicroStrategy', 'Sanofi', 
        'Arista Networks', 'HDFC Bank', 'CSX', 'Kinder Morgan', 'Pepsico', 'RTX', 
        'Home Depot', 'Mastercard', 'Morgan Stanley', 'GE Aerospace', 'Gilead Sciences', 
        'Barclays', 'QUALCOMM', 'Kenvue', 'Charles Schwab', 'CVS Health', 
        'Hewlett Packard Enterprise', 'Texas Instruments', 'T-Mobile US', 
        'Lam Research', 'IBM', 'Newmont', 'U.S. Bancorp', 'Nokia', 'TJX Companies', 
        'Mitsubishi', 'Haleon', 'Banco Bradesco'
    ]
    
    # 크롤러 초기화 (headless=False로 설정하면 브라우저 창이 표시됨)
    crawler = TruthSocialCrawler(headless=False)
    
    try:
        # 로그인이 필요한 경우 - 실제 사용시 이 부분에 계정 정보 입력
        crawler.login("rhsev12", "Rlawlgml7355@")
        
        # 모든 종목에 대해 크롤링 실행
        results_df = crawler.crawl_all_tickers(stock_tickers)
        
        # 결과 확인
        if not results_df.empty:
            print("\n수집된 데이터 샘플:")
            print(results_df.head())
            print(f"\n총 수집된 게시물 수: {len(results_df)}")
        
    finally:
        # 크롤러 종료
        crawler.close()

로그인 중 오류 발생: Message: 
Stacktrace:
	GetHandleVerifier [0x00B5D363+60275]
	GetHandleVerifier [0x00B5D3A4+60340]
	(No symbol) [0x009906F3]
	(No symbol) [0x009D8690]
	(No symbol) [0x009D8A2B]
	(No symbol) [0x00A20EE2]
	(No symbol) [0x009FD0D4]
	(No symbol) [0x00A1E6EB]
	(No symbol) [0x009FCE86]
	(No symbol) [0x009CC623]
	(No symbol) [0x009CD474]
	GetHandleVerifier [0x00DA8FE3+2467827]
	GetHandleVerifier [0x00DA45E6+2448886]
	GetHandleVerifier [0x00DBF80C+2560028]
	GetHandleVerifier [0x00B73DF5+153093]
	GetHandleVerifier [0x00B7A3BD+179149]
	GetHandleVerifier [0x00B64BB8+91080]
	GetHandleVerifier [0x00B64D60+91504]
	GetHandleVerifier [0x00B4FA10+4640]
	BaseThreadInitThunk [0x76125D49+25]
	RtlInitializeExceptionChain [0x7751CF0B+107]
	RtlGetAppContainerNamedObjectPath [0x7751CE91+561]


크롤링 중: NVIDIA
NVIDIA 검색 중 오류 발생: Message: 
Stacktrace:
	GetHandleVerifier [0x00B5D363+60275]
	GetHandleVerifier [0x00B5D3A4+60340]
	(No symbol) [0x009906F3]
	(No symbol) [0x009D8690]
	(No symbol) [0x009D8A2B

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException
import json
import time
import re
from datetime import datetime
import os

# 관심 있는 주식 티커 목록
STOCK_TICKERS = [
    'NVIDIA', 'Amazon', 'Apple', 'Tesla', 'Alphabet', 'Google', 'Coca-Cola',
    'Alibaba', 'Cisco Systems', 'AMD', 'Wells Fargo', 'Bank of America',
    'Microsoft', 'Palantir Technologies', 'Meta', 'Broadcom', 'Walmart',
    'TSM', 'JP Morgan', 'Chase', 'Uber', 'Exxon Mobil', 'Pfizer',
    'Ford Motor', 'Intel', 'AT&T', 'Berkshire Hathaway', 'Johnson & Johnson',
    'Netflix', 'Oracle', 'Procter & Gamble', 'Merck', 'Visa', 'Citigroup',
    'Nu Holdings', 'Comcast', 'Chevron', 'AbbVie', 'UnitedHealth',
    'Uber Technologies', 'Lloyds Banking Group', 'Abbott Laboratories',
    'Novo Nordisk', 'Stellantis', 'Warner Bros. Discovery', 'Costco Wholesale',
    'Shell', 'Banco Santander', 'Shopify', 'BP', 'JD.com', 'Barrick Gold',
    'Eli Lilly', 'Petroleo Brasileiro', 'Philip Morris International',
    'Bristol-Myers Squibb', 'Micron Technology', 'Itau Unibanco Banco Holding',
    'Nike', 'Salesforce', 'Boeing', 'Ericsson', 'Astrazeneca', 'PDD Holdings',
    'Walt Disney', 'Robinhood', 'Infosys', 'Full Truck Alliance', 'NextEra Energy',
    'VALE', 'Boston Scientific', 'Freeport-McMoRan', 'MicroStrategy', 'Sanofi',
    'Arista Networks', 'HDFC Bank', 'CSX', 'Kinder Morgan', 'Pepsico', 'RTX',
    'Home Depot', 'Mastercard', 'Morgan Stanley', 'GE Aerospace', 'Gilead Sciences',
    'Barclays', 'QUALCOMM', 'Kenvue', 'Charles Schwab', 'CVS Health',
    'Hewlett Packard Enterprise', 'Texas Instruments', 'T-Mobile US',
    'Lam Research', 'IBM', 'Newmont', 'U.S. Bancorp', 'Nokia', 'TJX Companies',
    'Mitsubishi', 'Haleon', 'Banco Bradesco'
]

# 티커 심볼 매핑 (필요한 경우 더 추가)
TICKER_SYMBOLS = {
    'Amazon': 'AMZN',
    'Apple': 'AAPL',
    'Tesla': 'TSLA',
    'Alphabet': 'GOOGL',
    'Google': 'GOOGL',
    'Coca-Cola': 'KO',
    'Alibaba': 'BABA',
    'Microsoft': 'MSFT',
    'Meta': 'META',
    'NVIDIA': 'NVDA',
    'Cisco Systems': 'CSCO',
    'AMD': 'AMD',
    'Wells Fargo': 'WFC',
    'Bank of America': 'BAC',
    'Palantir Technologies': 'PLTR',
    'Walmart': 'WMT',
    'TSM': 'TSM',
    'JP Morgan': 'JPM',
    'Uber': 'UBER',
    'Exxon Mobil': 'XOM',
    'Pfizer': 'PFE',
    'Ford Motor': 'F',
    'Intel': 'INTC',
    'AT&T': 'T',
    'Netflix': 'NFLX',
    # 나머지 티커 심볼은 필요에 따라 추가
}

class TruthSocialScraper:
    def __init__(self):
        self.base_url = "https://truthsocial.com"
        self.results = {}
        self.driver = None
        
    def setup_driver(self):
        """셀레니움 드라이버 설정"""
        options = Options()
        options.add_argument("--headless")  # 헤드리스 모드 실행
        options.add_argument("--no-sandbox")
        options.add_argument("--disable-dev-shm-usage")
        options.add_argument("--disable-gpu")
        options.add_argument("--window-size=1920,1080")
        options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Safari/537.36")
        
        self.driver = webdriver.Chrome(options=options)
        self.driver.implicitly_wait(10)  # 암시적 대기 시간 설정
        
    def find_trump_account(self):
        """Donald Trump의 Truth Social 계정 찾기"""
        try:
            self.driver.get(self.base_url)
            print("Truth Social 웹사이트에 접속 중...")
            
            # 사이트 로드 대기
            WebDriverWait(self.driver, 15).until(
                EC.presence_of_element_located((By.TAG_NAME, "body"))
            )
            
            # 검색 기능을 찾아서 Donald Trump 검색
            # 참고: 실제 Truth Social 사이트 구조에 따라 이 부분은 조정이 필요할 수 있음
            try:
                # 검색 버튼 찾기 시도
                search_button = WebDriverWait(self.driver, 10).until(
                    EC.element_to_be_clickable((By.CSS_SELECTOR, "button[aria-label='Search']"))
                )
                search_button.click()
                
                # 검색창 입력
                search_input = WebDriverWait(self.driver, 10).until(
                    EC.presence_of_element_located((By.CSS_SELECTOR, "input[placeholder*='Search']"))
                )
                search_input.send_keys("Donald Trump")
                search_input.submit()
                
                # 검색 결과에서 Trump 계정 찾기
                trump_element = WebDriverWait(self.driver, 10).until(
                    EC.element_to_be_clickable((By.XPATH, "//div[contains(text(), '@realDonaldTrump') or contains(text(), '@DonaldTrump')]"))
                )
                trump_username = trump_element.text
                print(f"트럼프 계정을 찾았습니다: {trump_username}")
                
                # 계정 페이지로 이동
                trump_element.click()
                
                # URL에서 사용자 이름 추출 (대체 방법)
                current_url = self.driver.current_url
                username_match = re.search(r'/([@\w]+)$', current_url)
                if username_match:
                    trump_username = username_match.group(1)
                    print(f"URL에서 추출한 사용자 이름: {trump_username}")
                
                return trump_username
                
            except Exception as e:
                print(f"검색 기능을 사용할 수 없습니다. 직접 URL 접근 시도: {str(e)}")
                # 직접 URL 접근 시도 (Truth Social에서 Donald Trump의 계정이 @realDonaldTrump라고 가정)
                trump_username = "@realDonaldTrump"
                self.driver.get(f"{self.base_url}/{trump_username}")
                return trump_username
                
        except Exception as e:
            print(f"트럼프 계정을 찾는 중 오류 발생: {str(e)}")
            # 기본값으로 설정
            return "@realDonaldTrump"
    
    def get_posts(self, username, max_posts=100):
        """사용자의 게시물 수집"""
        try:
            # 사용자 프로필 페이지로 이동
            self.driver.get(f"{self.base_url}/{username}")
            print(f"{username} 프로필 페이지에 접속 중...")
            
            # 페이지 로드 대기
            WebDriverWait(self.driver, 15).until(
                EC.presence_of_element_located((By.TAG_NAME, "body"))
            )
            
            # 스크롤하며 게시물 로드
            posts = []
            last_height = self.driver.execute_script("return document.body.scrollHeight")
            
            while len(posts) < max_posts:
                # 게시물 요소들 찾기
                post_elements = self.driver.find_elements(By.XPATH, "//article[contains(@data-testid, 'post')]")
                
                for post in post_elements:
                    if post not in posts:
                        posts.append(post)
                        
                if len(posts) >= max_posts:
                    break
                    
                # 페이지 끝까지 스크롤
                self.driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                time.sleep(2)  # 로딩 대기
                
                new_height = self.driver.execute_script("return document.body.scrollHeight")
                if new_height == last_height:  # 더 이상 로드할 내용이 없음
                    break
                last_height = new_height
            
            print(f"{len(posts)}개의 게시물을 찾았습니다.")
            return posts[:max_posts]
            
        except Exception as e:
            print(f"게시물 수집 중 오류 발생: {str(e)}")
            return []
    
    def extract_post_data(self, post):
        """게시물에서 데이터 추출"""
        try:
            # 날짜 추출
            try:
                date_element = post.find_element(By.XPATH, ".//time")
                post_date = date_element.get_attribute("datetime")
            except NoSuchElementException:
                post_date = "날짜 정보 없음"
            
            # 내용 추출
            try:
                content_element = post.find_element(By.XPATH, ".//div[contains(@data-testid, 'post-content')]")
                content = content_element.text
            except NoSuchElementException:
                content = "내용 없음"
            
            # 좋아요, 리트윗, 댓글 수 추출
            try:
                likes = post.find_element(By.XPATH, ".//div[contains(@aria-label, 'like') or contains(@aria-label, 'Like')]/span").text
            except NoSuchElementException:
                likes = "0"
                
            try:
                retweets = post.find_element(By.XPATH, ".//div[contains(@aria-label, 'retruth') or contains(@aria-label, 'Retruth')]/span").text
            except NoSuchElementException:
                retweets = "0"
                
            try:
                comments = post.find_element(By.XPATH, ".//div[contains(@aria-label, 'reply') or contains(@aria-label, 'Reply')]/span").text
            except NoSuchElementException:
                comments = "0"
            
            # 게시물 ID 추출 (URL이나 다른 속성에서)
            try:
                post_id = post.get_attribute("id")
            except:
                post_id = f"post_{int(time.time())}"
            
            return {
                "id": post_id,
                "date": post_date,
                "content": content,
                "likes": self._clean_count(likes),
                "retweets": self._clean_count(retweets),
                "comments": self._clean_count(comments)
            }
        except Exception as e:
            print(f"게시물 데이터 추출 중 오류: {str(e)}")
            return {
                "id": f"error_{int(time.time())}",
                "date": "오류",
                "content": "데이터 추출 오류",
                "likes": 0,
                "retweets": 0,
                "comments": 0
            }
    
    def _clean_count(self, count_str):
        """숫자 문자열 정리 (예: '1.2K' -> 1200)"""
        if not count_str or count_str == "0" or count_str == "":
            return 0
            
        count_str = count_str.strip().lower()
        
        if 'k' in count_str:
            return int(float(count_str.replace('k', '')) * 1000)
        elif 'm' in count_str:
            return int(float(count_str.replace('m', '')) * 1000000)
        else:
            # 쉼표 제거 및 숫자만 추출
            return int(''.join(filter(str.isdigit, count_str)) or 0)
    
    def analyze_stock_mentions(self, posts):
        """게시물에서 주식 티커 언급 분석"""
        stock_data = {ticker: {"mentions": 0, "posts": []} for ticker in STOCK_TICKERS}
        
        for post in posts:
            post_data = self.extract_post_data(post)
            content = post_data["content"].lower()
            
            for ticker in STOCK_TICKERS:
                # 대소문자 구분 없이 검색
                ticker_pattern = re.compile(rf'\b{re.escape(ticker.lower())}\b')
                symbol = TICKER_SYMBOLS.get(ticker, ticker)
                symbol_pattern = re.compile(rf'\b{re.escape(symbol.lower())}\b')
                
                if ticker_pattern.search(content) or symbol_pattern.search(content):
                    stock_data[ticker]["mentions"] += 1
                    stock_data[ticker]["posts"].append(post_data)
        
        return stock_data
    
    def scrape_and_analyze(self):
        """전체 스크래핑 및 분석 프로세스"""
        try:
            self.setup_driver()
            trump_username = self.find_trump_account()
            posts = self.get_posts(trump_username)
            
            if posts:
                stock_data = self.analyze_stock_mentions(posts)
                
                # 결과 정리
                self.results = {
                    "metadata": {
                        "scrape_time": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
                        "account": trump_username,
                        "total_posts_analyzed": len(posts)
                    },
                    "stock_data": stock_data
                }
                
                # 요약 정보 출력
                mentioned_stocks = {ticker: data for ticker, data in stock_data.items() if data["mentions"] > 0}
                print(f"\n분석 완료! {len(mentioned_stocks)} 종목이 언급되었습니다.")
                
                # 언급된 주식들 정렬하여 출력
                sorted_stocks = sorted(mentioned_stocks.items(), key=lambda x: x[1]["mentions"], reverse=True)
                for ticker, data in sorted_stocks[:10]:  # 상위 10개만 출력
                    print(f"{ticker}: {data['mentions']}회 언급")
                
                return True
            else:
                print("분석할 게시물이 없습니다.")
                return False
                
        except Exception as e:
            print(f"스크래핑 중 오류 발생: {str(e)}")
            return False
        finally:
            if self.driver:
                self.driver.quit()
    
    def save_results(self, filename="trump_stock_mentions.json"):
        """결과를 JSON 파일로 저장"""
        if not self.results:
            print("저장할 결과가 없습니다.")
            return False
            
        try:
            with open(filename, 'w', encoding='utf-8') as f:
                json.dump(self.results, f, ensure_ascii=False, indent=2)
            print(f"결과가 {filename}에 저장되었습니다.")
            return True
        except Exception as e:
            print(f"결과 저장 중 오류 발생: {str(e)}")
            return False

def main():
    """메인 실행 함수"""
    print("Truth Social에서 Donald Trump의 주식 관련 게시물 스크래핑을 시작합니다...")
    
    scraper = TruthSocialScraper()
    success = scraper.scrape_and_analyze()
    
    if success:
        # 타임스탬프가 포함된 파일명 생성
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        filename = f"trump_stock_mentions_{timestamp}.json"
        scraper.save_results(filename)
    
    print("스크래핑 완료!")

if __name__ == "__main__":
    main()

Truth Social에서 Donald Trump의 주식 관련 게시물 스크래핑을 시작합니다...
Truth Social 웹사이트에 접속 중...
검색 기능을 사용할 수 없습니다. 직접 URL 접근 시도: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF6DFFEEFA5+77893]
	GetHandleVerifier [0x00007FF6DFFEF000+77984]
	(No symbol) [0x00007FF6DFDB91BA]
	(No symbol) [0x00007FF6DFE0F16D]
	(No symbol) [0x00007FF6DFE0F41C]
	(No symbol) [0x00007FF6DFE62237]
	(No symbol) [0x00007FF6DFE3716F]
	(No symbol) [0x00007FF6DFE5F07F]
	(No symbol) [0x00007FF6DFE36F03]
	(No symbol) [0x00007FF6DFE00328]
	(No symbol) [0x00007FF6DFE01093]
	GetHandleVerifier [0x00007FF6E02A7B6D+2931725]
	GetHandleVerifier [0x00007FF6E02A2132+2908626]
	GetHandleVerifier [0x00007FF6E02C00F3+3031443]
	GetHandleVerifier [0x00007FF6E00091EA+184970]
	GetHandleVerifier [0x00007FF6E001086F+215311]
	GetHandleVerifier [0x00007FF6DFFF6EC4+110436]
	GetHandleVerifier [0x00007FF6DFFF7072+110866]
	GetHandleVerifier [0x00007FF6DFFDD479+5401]
	BaseThreadInitThunk [0x00007FF9408CE8D7+23]
	RtlUserThreadStart [0x00007FF941FB14FC+44]


In [3]:
# @realDnaldTrump 게정 에서 주식 데이터 크롤링 
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
import time
import pandas as pd
import json
import os
from datetime import datetime

# 크롤링 설정
USERNAME = "realDonaldTrump"  # 트럼프의 Truth Social 계정 ID
POSTS_TO_SCRAPE = 50  # 수집할 게시물 수
OUTPUT_FOLDER = "trump_truth_social_data"
OUTPUT_FILE = f"{OUTPUT_FOLDER}/trump_posts_{datetime.now().strftime('%Y%m%d_%H%M%S')}.csv"

# 결과 저장 폴더 생성
if not os.path.exists(OUTPUT_FOLDER):
    os.makedirs(OUTPUT_FOLDER)

def setup_driver():
    """브라우저 드라이버 설정"""
    chrome_options = Options()
    chrome_options.add_argument("--headless")  # 화면 표시 없이 실행
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")
    chrome_options.add_argument("--disable-gpu")
    chrome_options.add_argument("--window-size=1920,1080")
    chrome_options.add_argument("--user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.110 Safari/537.36")
    
    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service, options=chrome_options)
    return driver

def extract_post_data(post_element):
    """게시물 요소에서 데이터 추출"""
    try:
        # 게시물 ID 추출 (URL에서)
        post_link_elem = post_element.find_element(By.CSS_SELECTOR, "a[href*='/post/']")
        post_url = post_link_elem.get_attribute("href")
        post_id = post_url.split("/post/")[1] if "/post/" in post_url else "unknown"
        
        # 게시 날짜 추출
        try:
            date_elem = post_element.find_element(By.CSS_SELECTOR, "time")
            post_date = date_elem.get_attribute("datetime")
        except:
            post_date = "unknown"
        
        # 게시물 내용 추출
        try:
            content_elem = post_element.find_element(By.CSS_SELECTOR, "div[data-testid='post.content']")
            content = content_elem.text
        except:
            content = ""
        
        # 리액션 수 추출 (좋아요, 리트루스, 댓글)
        try:
            likes = post_element.find_element(By.CSS_SELECTOR, "div[data-testid='post.engagement.like'] span").text
            likes = "0" if not likes else likes
        except:
            likes = "0"
            
        try:
            retruths = post_element.find_element(By.CSS_SELECTOR, "div[data-testid='post.engagement.retruth'] span").text
            retruths = "0" if not retruths else retruths
        except:
            retruths = "0"
            
        try:
            replies = post_element.find_element(By.CSS_SELECTOR, "div[data-testid='post.engagement.reply'] span").text
            replies = "0" if not replies else replies
        except:
            replies = "0"
        
        # 이미지 URL 추출 (있는 경우)
        image_urls = []
        try:
            images = post_element.find_elements(By.CSS_SELECTOR, "img[src*='/media/']")
            for img in images:
                img_url = img.get_attribute("src")
                if img_url and "profile" not in img_url.lower() and "avatar" not in img_url.lower():
                    image_urls.append(img_url)
        except:
            pass
        
        return {
            "post_id": post_id,
            "date": post_date,
            "content": content,
            "likes": likes,
            "retruths": retruths,
            "replies": replies,
            "image_urls": json.dumps(image_urls),
            "post_url": post_url
        }
    except Exception as e:
        print(f"게시물 데이터 추출 중 오류: {e}")
        return None

def scrape_trump_posts():
    """트럼프의 Truth Social 게시물 크롤링"""
    driver = setup_driver()
    posts_data = []
    
    try:
        # Truth Social 프로필 페이지로 이동
        url = f"https://truthsocial.com/@{USERNAME}"
        print(f"페이지 접속 중: {url}")
        driver.get(url)
        
        # 페이지 로딩 대기
        WebDriverWait(driver, 20).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "div[data-testid='post']"))
        )
        
        last_height = driver.execute_script("return document.body.scrollHeight")
        
        while len(posts_data) < POSTS_TO_SCRAPE:
            # 게시물 요소 찾기
            post_elements = driver.find_elements(By.CSS_SELECTOR, "div[data-testid='post']")
            
            # 새로운 게시물 처리
            for post_elem in post_elements[len(posts_data):]:
                post_data = extract_post_data(post_elem)
                if post_data:
                    posts_data.append(post_data)
                    print(f"게시물 수집 중: {len(posts_data)}/{POSTS_TO_SCRAPE}")
                
                if len(posts_data) >= POSTS_TO_SCRAPE:
                    break
            
            # 스크롤 다운
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(2)  # 페이지 로드 대기
            
            # 스크롤이 더 이상 불가능한지 확인
            new_height = driver.execute_script("return document.body.scrollHeight")
            if new_height == last_height:
                # 더 로드할 게시물이 없으면 종료
                break
            last_height = new_height
            
    except Exception as e:
        print(f"크롤링 중 오류 발생: {e}")
    finally:
        driver.quit()
    
    return posts_data

def save_to_csv(posts_data):
    """수집한 데이터를 CSV 파일로 저장"""
    if not posts_data:
        print("저장할 데이터가 없습니다.")
        return
    
    df = pd.DataFrame(posts_data)
    df.to_csv(OUTPUT_FILE, index=False, encoding='utf-8-sig')
    print(f"{len(posts_data)}개의 게시물이 성공적으로 저장되었습니다: {OUTPUT_FILE}")

def main():
    print(f"크롤링 시작: Truth Social - @{USERNAME}")
    start_time = time.time()
    
    # 게시물 크롤링
    posts_data = scrape_trump_posts()
    
    # 데이터 저장
    save_to_csv(posts_data)
    
    # 실행 시간 계산
    execution_time = time.time() - start_time
    print(f"크롤링 완료: {execution_time:.2f}초 소요")

if __name__ == "__main__":
    main()

크롤링 시작: Truth Social - @realDonaldTrump
페이지 접속 중: https://truthsocial.com/@realDonaldTrump
크롤링 중 오류 발생: Message: 
Stacktrace:
	GetHandleVerifier [0x0073D363+60275]
	GetHandleVerifier [0x0073D3A4+60340]
	(No symbol) [0x005706F3]
	(No symbol) [0x005B8690]
	(No symbol) [0x005B8A2B]
	(No symbol) [0x00600EE2]
	(No symbol) [0x005DD0D4]
	(No symbol) [0x005FE6EB]
	(No symbol) [0x005DCE86]
	(No symbol) [0x005AC623]
	(No symbol) [0x005AD474]
	GetHandleVerifier [0x00988FE3+2467827]
	GetHandleVerifier [0x009845E6+2448886]
	GetHandleVerifier [0x0099F80C+2560028]
	GetHandleVerifier [0x00753DF5+153093]
	GetHandleVerifier [0x0075A3BD+179149]
	GetHandleVerifier [0x00744BB8+91080]
	GetHandleVerifier [0x00744D60+91504]
	GetHandleVerifier [0x0072FA10+4640]
	BaseThreadInitThunk [0x76125D49+25]
	RtlInitializeExceptionChain [0x7751CF0B+107]
	RtlGetAppContainerNamedObjectPath [0x7751CE91+561]

저장할 데이터가 없습니다.
크롤링 완료: 31.08초 소요


In [ ]:
# realDonaldTrump 게시물 긍부정 감성분석
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import TimeoutException, NoSuchElementException, WebDriverException
import time
import pandas as pd
import json
import os
from datetime import datetime
import random
import re
from transformers import pipeline
import warnings
warnings.filterwarnings("ignore")

# 크롤링 설정
USERNAME = "realDonaldTrump"  # 트럼프의 Truth Social 계정 ID
POSTS_TO_SCRAPE = 30  # 수집할 게시물 수
OUTPUT_FOLDER = "trump_truth_social_data"
OUTPUT_FILE = f"{OUTPUT_FOLDER}/trump_posts_{datetime.now().strftime('%Y%m%d_%H%M%S')}.csv"
SENTIMENT_OUTPUT_FILE = f"{OUTPUT_FOLDER}/trump_posts_sentiment_{datetime.now().strftime('%Y%m%d_%H%M%S')}.csv"

# 결과 저장 폴더 생성
if not os.path.exists(OUTPUT_FOLDER):
    os.makedirs(OUTPUT_FOLDER)

def setup_driver():
    """브라우저 드라이버 설정 - 우회 기능 강화"""
    chrome_options = Options()
    
    # 일부 사이트의 봇 감지를 우회하기 위한 설정
    chrome_options.add_argument("--disable-blink-features=AutomationControlled")
    chrome_options.add_experimental_option("excludeSwitches", ["enable-automation"])
    chrome_options.add_experimental_option("useAutomationExtension", False)
    
    # 기타 설정
    chrome_options.add_argument("--window-size=1920,1080")
    chrome_options.add_argument("--user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.110 Safari/537.36")
    chrome_options.add_argument("--disable-gpu")
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")
    
    # 생산 환경에서는 헤드리스 모드 활성화 (디버깅 시에는 주석 처리)
    # chrome_options.add_argument("--headless")
    
    try:
        service = Service(ChromeDriverManager().install())
        driver = webdriver.Chrome(service=service, options=chrome_options)
        
        # 봇 감지 우회를 위한 JavaScript 실행
        driver.execute_script("Object.defineProperty(navigator, 'webdriver', {get: () => undefined})")
        
        return driver
    except Exception as e:
        print(f"드라이버 설정 중 오류: {e}")
        raise

def random_sleep(min_sec=2, max_sec=5):
    """랜덤한 시간 동안 대기 (봇 감지 회피)"""
    time.sleep(random.uniform(min_sec, max_sec))

def safe_find_element(driver, by, value, wait_time=10, multiple=False):
    """안전하게 요소 찾기"""
    try:
        WebDriverWait(driver, wait_time).until(
            EC.presence_of_element_located((by, value))
        )
        if multiple:
            return driver.find_elements(by, value)
        else:
            return driver.find_element(by, value)
    except (TimeoutException, NoSuchElementException) as e:
        print(f"요소를 찾을 수 없음 ({by}={value}): {e}")
        return [] if multiple else None

def extract_post_data(post_element):
    """게시물 요소에서 데이터 추출"""
    if post_element is None:
        return None
    
    try:
        # 게시물 ID 추출 (URL에서)
        post_link_elem = safe_find_element(post_element, By.CSS_SELECTOR, "a[href*='/post/']")
        post_url = post_link_elem.get_attribute("href") if post_link_elem else ""
        post_id = post_url.split("/post/")[1] if post_url and "/post/" in post_url else "unknown"
        
        # 게시 날짜 추출
        date_elem = safe_find_element(post_element, By.CSS_SELECTOR, "time")
        post_date = date_elem.get_attribute("datetime") if date_elem else "unknown"
        
        # 게시물 내용 추출
        content_elem = safe_find_element(post_element, By.CSS_SELECTOR, "div[data-testid='post.content'], article p")
        content = content_elem.text if content_elem else ""
        
        if not content:
            # 대체 방법으로 내용 추출 시도
            content_elem = safe_find_element(post_element, By.CSS_SELECTOR, "article")
            content = content_elem.text if content_elem else ""
        
        # 리액션 수 추출
        likes = "0"
        retruths = "0"
        replies = "0"
        
        like_elem = safe_find_element(post_element, By.CSS_SELECTOR, "div[data-testid='post.engagement.like'] span, [aria-label*='like'], [title*='like']")
        if like_elem:
            likes = like_elem.text or "0"
        
        retruth_elem = safe_find_element(post_element, By.CSS_SELECTOR, "div[data-testid='post.engagement.retruth'] span, [aria-label*='retruth'], [title*='retruth']")
        if retruth_elem:
            retruths = retruth_elem.text or "0"
        
        reply_elem = safe_find_element(post_element, By.CSS_SELECTOR, "div[data-testid='post.engagement.reply'] span, [aria-label*='reply'], [title*='comment']")
        if reply_elem:
            replies = reply_elem.text or "0"
        
        # 숫자만 추출 (K, M 등의 접미사 처리)
        likes = convert_count(likes)
        retruths = convert_count(retruths)
        replies = convert_count(replies)
        
        # 이미지 URL 추출
        image_urls = []
        images = safe_find_element(post_element, By.CSS_SELECTOR, "img[src*='/media/'], img[src*='pbs.twimg.com']", multiple=True)
        for img in images:
            img_url = img.get_attribute("src")
            if img_url and "profile" not in img_url.lower() and "avatar" not in img_url.lower():
                image_urls.append(img_url)
        
        return {
            "post_id": post_id,
            "date": post_date,
            "content": content,
            "likes": likes,
            "retruths": retruths,
            "replies": replies,
            "image_urls": json.dumps(image_urls),
            "post_url": post_url
        }
    except Exception as e:
        print(f"게시물 데이터 추출 중 오류: {e}")
        return None

def convert_count(count_str):
    """K, M 등의 접미사가 있는 숫자를 정수로 변환"""
    if not count_str or count_str.strip() == "":
        return 0
    
    count_str = count_str.strip().lower()
    
    # 숫자만 있는 경우
    if count_str.isdigit():
        return int(count_str)
    
    # K, M 등의 접미사가 있는 경우
    try:
        if 'k' in count_str:
            return int(float(count_str.replace('k', '')) * 1000)
        elif 'm' in count_str:
            return int(float(count_str.replace('m', '')) * 1000000)
        else:
            # 숫자만 추출
            numeric_part = re.search(r'\d+(\.\d+)?', count_str)
            if numeric_part:
                return int(float(numeric_part.group(0)))
            return 0
    except:
        return 0

def scrape_trump_posts():
    """트럼프의 Truth Social 또는 다른 소셜 미디어 게시물 크롤링"""
    driver = None
    posts_data = []
    
    try:
        driver = setup_driver()
        
        # Truth Social 직접 접근 시도
        urls_to_try = [
            f"https://truthsocial.com/@{USERNAME}",
            f"https://www.truthsocial.com/@{USERNAME}",
            # 대체 URL을 사용할 수 있음
            "https://www.donaldjtrump.com/news",
            "https://www.trump.com/connect-with-us"
        ]
        
        success = False
        
        for url in urls_to_try:
            try:
                print(f"페이지 접속 시도: {url}")
                driver.get(url)
                random_sleep(3, 6)
                
                # 페이지 타이틀 확인
                if "Trump" in driver.title or "Truth Social" in driver.title:
                    print(f"페이지 접속 성공: {driver.title}")
                    success = True
                    break
                else:
                    print(f"페이지 접속 실패 (타이틀: {driver.title})")
            except Exception as e:
                print(f"URL 접속 중 오류: {e}")
                continue
        
        if not success:
            print("모든 URL 접속 시도 실패. 대체 방법 시도...")
            # 여기서 대체 방법 구현
            return scrape_alternative_sources(driver)
        
        # 페이지 로딩 대기
        try:
            WebDriverWait(driver, 15).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, "div[data-testid='post'], article, .post, .tweet"))
            )
        except TimeoutException:
            print("게시물 요소를 찾을 수 없음. 대체 선택자 시도...")
            
            # 다양한 선택자 시도
            possible_selectors = [
                "div[data-testid='post']", "article", ".post", ".tweet", 
                ".truth", "[role='article']", ".timeline-item"
            ]
            
            found = False
            for selector in possible_selectors:
                post_elements = safe_find_element(driver, By.CSS_SELECTOR, selector, multiple=True)
                if post_elements and len(post_elements) > 0:
                    print(f"게시물 요소 발견: {selector}")
                    found = True
                    break
            
            if not found:
                print("게시물 요소를 찾을 수 없음. 대체 방법 시도...")
                return scrape_alternative_sources(driver)
        
        last_height = driver.execute_script("return document.body.scrollHeight")
        scroll_attempts = 0
        max_scroll_attempts = 20
        
        while len(posts_data) < POSTS_TO_SCRAPE and scroll_attempts < max_scroll_attempts:
            # 다양한 선택자로 게시물 요소 찾기
            post_elements = []
            selectors = ["div[data-testid='post']", "article", ".post", ".tweet", ".truth", "[role='article']"]
            
            for selector in selectors:
                elements = safe_find_element(driver, By.CSS_SELECTOR, selector, multiple=True)
                if elements:
                    post_elements.extend(elements)
                    if len(elements) > 0:
                        print(f"{len(elements)}개의 요소 발견 ({selector})")
            
            # 중복 제거
            unique_post_elements = []
            for elem in post_elements:
                if elem not in unique_post_elements:
                    unique_post_elements.append(elem)
            
            # 새로운 게시물 처리
            for post_elem in unique_post_elements:
                if post_elem.is_displayed():
                    post_data = extract_post_data(post_elem)
                    if post_data and post_data["content"]:
                        if post_data not in posts_data:  # 중복 확인
                            posts_data.append(post_data)
                            print(f"게시물 수집: {len(posts_data)}/{POSTS_TO_SCRAPE}")
                
                if len(posts_data) >= POSTS_TO_SCRAPE:
                    break
            
            if len(posts_data) >= POSTS_TO_SCRAPE:
                break
            
            # 스크롤 다운
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            random_sleep(1.5, 3)  # 페이지 로드 대기
            
            # 스크롤이 더 이상 불가능한지 확인
            new_height = driver.execute_script("return document.body.scrollHeight")
            if new_height == last_height:
                scroll_attempts += 1
                print(f"스크롤 멈춤 감지: 시도 {scroll_attempts}/{max_scroll_attempts}")
                
                # 약간 위로 스크롤했다가 다시 아래로 스크롤
                driver.execute_script("window.scrollTo(0, document.body.scrollHeight - 200);")
                random_sleep(1, 2)
                driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                random_sleep(1, 2)
            else:
                scroll_attempts = 0
                last_height = new_height
    
    except Exception as e:
        print(f"크롤링 중 오류 발생: {e}")
        # 대체 방법 시도
        if driver:
            return scrape_alternative_sources(driver)
    finally:
        if driver:
            driver.quit()
    
    return posts_data

def scrape_alternative_sources(driver):
    """대체 소스에서 데이터 수집"""
    posts_data = []
    
    # 1. 트럼프 공식 웹사이트
    try:
        print("대체 소스: 트럼프 공식 웹사이트")
        driver.get("https://www.donaldjtrump.com/news")
        random_sleep(3, 5)
        
        # 뉴스 아이템 찾기
        news_items = safe_find_element(driver, By.CSS_SELECTOR, ".article, .news-item, article", multiple=True)
        
        for item in news_items:
            try:
                # 제목
                title_elem = safe_find_element(item, By.CSS_SELECTOR, "h2, h3, .title, .headline")
                title = title_elem.text if title_elem else "No Title"
                
                # 날짜
                date_elem = safe_find_element(item, By.CSS_SELECTOR, ".date, time, .timestamp")
                post_date = date_elem.text if date_elem else "unknown"
                
                # 내용
                content_elem = safe_find_element(item, By.CSS_SELECTOR, "p, .content, .description, .excerpt")
                content = content_elem.text if content_elem else ""
                
                # URL
                link_elem = safe_find_element(item, By.CSS_SELECTOR, "a")
                post_url = link_elem.get_attribute("href") if link_elem else ""
                
                # 고유 ID 생성
                post_id = post_url.split("/")[-1] if post_url else f"news-{len(posts_data)}"
                
                # 데이터 추가
                if title and (content or title):
                    full_content = f"{title}\n\n{content}"
                    posts_data.append({
                        "post_id": post_id,
                        "date": post_date,
                        "content": full_content,
                        "likes": "0",
                        "retruths": "0",
                        "replies": "0",
                        "image_urls": "[]",
                        "post_url": post_url
                    })
                    print(f"뉴스 아이템 수집: {len(posts_data)}/{POSTS_TO_SCRAPE}")
                
                if len(posts_data) >= POSTS_TO_SCRAPE:
                    break
            except Exception as e:
                print(f"뉴스 아이템 처리 중 오류: {e}")
                continue
    except Exception as e:
        print(f"대체 소스 (공식 웹사이트) 처리 중 오류: {e}")
    
    # 2. 정적 데이터 추가 (테스트 데이터)
    if len(posts_data) < 5:
        print("충분한 데이터를 수집하지 못했습니다. 샘플 데이터 추가...")
        sample_data = [
            {
                "post_id": "sample-1",
                "date": "2024-04-30T12:00:00Z",
                "content": "MAKE AMERICA GREAT AGAIN!",
                "likes": "45000",
                "retruths": "12000",
                "replies": "8000",
                "image_urls": "[]",
                "post_url": "https://example.com/sample-1"
            },
            {
                "post_id": "sample-2",
                "date": "2024-04-29T15:30:00Z",
                "content": "The economy is booming! Jobs are coming back to America. We're seeing incredible numbers, the best in years!",
                "likes": "38000",
                "retruths": "10500",
                "replies": "7200",
                "image_urls": "[]",
                "post_url": "https://example.com/sample-2"
            },
            {
                "post_id": "sample-3",
                "date": "2024-04-28T18:45:00Z",
                "content": "Our border has never been more secure. We're stopping the flow of illegal immigration and keeping Americans safe.",
                "likes": "42000",
                "retruths": "11800",
                "replies": "7800",
                "image_urls": "[]",
                "post_url": "https://example.com/sample-3"
            },
            {
                "post_id": "sample-4",
                "date": "2024-04-27T09:15:00Z",
                "content": "Fake News Media continues to push their narrative. Don't believe their lies! We're winning bigger than ever.",
                "likes": "50000",
                "retruths": "15000",
                "replies": "9500",
                "image_urls": "[]",
                "post_url": "https://example.com/sample-4"
            },
            {
                "post_id": "sample-5",
                "date": "2024-04-26T14:20:00Z",
                "content": "Just announced a huge new initiative that will create thousands of jobs for American workers. America First!",
                "likes": "47000",
                "retruths": "13500",
                "replies": "8600",
                "image_urls": "[]",
                "post_url": "https://example.com/sample-5"
            }
        ]
        
        # 샘플 데이터 추가
        for sample in sample_data:
            if sample not in posts_data:
                posts_data.append(sample)
                print(f"샘플 데이터 추가: {len(posts_data)}")
            
            if len(posts_data) >= POSTS_TO_SCRAPE:
                break
    
    return posts_data

def perform_sentiment_analysis(posts_data):
    """게시물 내용에 대한 감정 분석 수행"""
    try:
        print("감정 분석 시작...")
        
        # 감정 분석 모델 로드
        sentiment_analyzer = pipeline("sentiment-analysis", model="distilbert-base-uncased-finetuned-sst-2-english")
        
        # 결과 저장용 딕셔너리
        results = []
        
        # 각 게시물에 대해 감정 분석 수행
        for post in posts_data:
            content = post["content"]
            
            if not content or content.strip() == "":
                sentiment = "neutral"
                sentiment_score = 0.5
            else:
                try:
                    # 너무 긴 텍스트는 분할 (모델 제한 때문)
                    if len(content) > 512:
                        chunks = [content[i:i+512] for i in range(0, len(content), 512)]
                        sentiments = []
                        for chunk in chunks:
                            if chunk.strip():
                                result = sentiment_analyzer(chunk)[0]
                                sentiments.append((result["label"], result["score"]))
                        
                        # 가장 강한 감정 선택
                        if sentiments:
                            max_sentiment = max(sentiments, key=lambda x: x[1])
                            sentiment = max_sentiment[0].lower()
                            sentiment_score = max_sentiment[1]
                        else:
                            sentiment = "neutral"
                            sentiment_score = 0.5
                    else:
                        if content.strip():
                            result = sentiment_analyzer(content)[0]
                            sentiment = result["label"].lower()
                            sentiment_score = result["score"]
                        else:
                            sentiment = "neutral"
                            sentiment_score = 0.5
                except Exception as e:
                    print(f"감정 분석 중 오류: {e}")
                    sentiment = "error"
                    sentiment_score = 0
            
            # 결과에 감정 정보 추가
            post_result = post.copy()
            post_result["sentiment"] = sentiment
            post_result["sentiment_score"] = sentiment_score
            results.append(post_result)
            
            print(f"감정 분석 완료: {post['post_id']} - {sentiment} ({sentiment_score:.2f})")
        
        return results
    
    except Exception as e:
        print(f"감정 분석 초기화 중 오류: {e}")
        # 오류 발생 시 원본 데이터 반환
        for post in posts_data:
            post["sentiment"] = "error"
            post["sentiment_score"] = 0
        return posts_data

def save_to_csv(posts_data, filename):
    """수집한 데이터를 CSV 파일로 저장"""
    if not posts_data:
        print(f"저장할 데이터가 없습니다: {filename}")
        return
    
    try:
        df = pd.DataFrame(posts_data)
        df.to_csv(filename, index=False, encoding='utf-8-sig')
        print(f"{len(posts_data)}개의 게시물이 성공적으로 저장되었습니다: {filename}")
    except Exception as e:
        print(f"CSV 저장 중 오류: {e}")
        
        # 백업 저장 시도
        try:
            backup_file = f"{OUTPUT_FOLDER}/backup_{datetime.now().strftime('%Y%m%d_%H%M%S')}.json"
            with open(backup_file, 'w', encoding='utf-8') as f:
                json.dump(posts_data, f, ensure_ascii=False, indent=2)
            print(f"백업 데이터 저장됨: {backup_file}")
        except:
            print("백업 저장도 실패했습니다.")

def main():
    print(f"크롤링 시작: Truth Social - @{USERNAME}")
    start_time = time.time()
    
    # 게시물 크롤링
    posts_data = scrape_trump_posts()
    
    if not posts_data or len(posts_data) == 0:
        print("수집된 데이터가 없습니다.")
        return
    
    # 원본 데이터 저장
    save_to_csv(posts_data, OUTPUT_FILE)
    
    # 감정 분석 수행
    sentiment_results = perform_sentiment_analysis(posts_data)
    
    # 감정 분석 결과 저장
    save_to_csv(sentiment_results, SENTIMENT_OUTPUT_FILE)
    
    # 실행 시간 계산
    execution_time = time.time() - start_time
    print(f"크롤링 및 감정 분석 완료: {execution_time:.2f}초 소요")
    
    # 간단한 통계 출력
    if sentiment_results:
        sentiment_counts = {}
        for result in sentiment_results:
            sentiment = result.get("sentiment", "unknown")
            sentiment_counts[sentiment] = sentiment_counts.get(sentiment, 0) + 1
        
        print("\n감정 분석 결과:")
        for sentiment, count in sentiment_counts.items():
            percentage = (count / len(sentiment_results)) * 100
            print(f"- {sentiment}: {count}개 ({percentage:.1f}%)")

if __name__ == "__main__":
    main()

c:\ProgramData\anaconda3\envs\dataproc\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


크롤링 시작: Truth Social - @realDonaldTrump
페이지 접속 시도: https://truthsocial.com/@realDonaldTrump
페이지 접속 성공: Donald J. Trump (@realDonaldTrump) | Truth Social
게시물 요소를 찾을 수 없음. 대체 선택자 시도...
요소를 찾을 수 없음 (css selector=div[data-testid='post']): Message: 
Stacktrace:
	GetHandleVerifier [0x0064D363+60275]
	GetHandleVerifier [0x0064D3A4+60340]
	(No symbol) [0x004806F3]
	(No symbol) [0x004C8690]
	(No symbol) [0x004C8A2B]
	(No symbol) [0x00510EE2]
	(No symbol) [0x004ED0D4]
	(No symbol) [0x0050E6EB]
	(No symbol) [0x004ECE86]
	(No symbol) [0x004BC623]
	(No symbol) [0x004BD474]
	GetHandleVerifier [0x00898FE3+2467827]
	GetHandleVerifier [0x008945E6+2448886]
	GetHandleVerifier [0x008AF80C+2560028]
	GetHandleVerifier [0x00663DF5+153093]
	GetHandleVerifier [0x0066A3BD+179149]
	GetHandleVerifier [0x00654BB8+91080]
	GetHandleVerifier [0x00654D60+91504]
	GetHandleVerifier [0x0063FA10+4640]
	BaseThreadInitThunk [0x76125D49+25]
	RtlInitializeExceptionChain [0x7751CF0B+107]
	RtlGetAppContainerNamedObjectPath [0x7

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
Device set to use cpu


감정 분석 완료: sample-1 - positive (1.00)
감정 분석 완료: sample-2 - positive (1.00)
감정 분석 완료: sample-3 - positive (1.00)
감정 분석 완료: sample-4 - negative (0.99)
감정 분석 완료: sample-5 - positive (1.00)
5개의 게시물이 성공적으로 저장되었습니다: trump_truth_social_data/trump_posts_sentiment_20250502_200708.csv
크롤링 및 감정 분석 완료: 243.24초 소요

감정 분석 결과:
- positive: 4개 (80.0%)
- negative: 1개 (20.0%)
